<a href="https://colab.research.google.com/github/AgarwalMayank2/Face_Detection/blob/main/applying_ML_algorithms/KNN_PRMLproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# KNN Model

In [2]:
import numpy as np
import math

In [3]:
def euclidean_dist(x,y):
  # x1,y1,z1=x
  # x2,y2,z2=y
  return math.sqrt(sum([(a-b)**2 for a,b in zip(x,y)]))

In [4]:
from collections import Counter

In [5]:
def knn(k, X_train, X_test, y_test):
  predictions=[]
  for i in range(X_test.shape[0]):
    distances=[]
    nearest_labels=[]
    for j in range(X_train.shape[0]):
      # append the pair (distance, index) to distances
      dist=euclidean_dist(X_test[i], X_train[j])
      distances.append((dist, j))
    # sorting distances on the basis of first item of pair i.e. distance
    distances.sort(key=lambda x:x[0])
    for k1 in range(min(k, len(distances))):  # Ensure k does not exceed available distances
      nearest_labels.append(y_train[distances[k1][1]])

    label_counts = Counter(nearest_labels)
    most_common_label = label_counts.most_common(1)[0][0] if nearest_labels else -1

    predictions.append(label_encoder.inverse_transform([most_common_label])[0])

    if (i<=100 and i%10==0) or i==200:
      print(f'Label {i} detected : {predictions[i]}')

  accuracy = findAccuracy(predictions, y_test)
  return predictions, accuracy

In [6]:
def findAccuracy(predictions, y_test):
  count=0
  print(predictions)
  # print(y_test)
  for i in range(len(predictions)):
    if predictions[i]==label_encoder.inverse_transform([y_test[i]])[0]:
      count+=1

  accuracy = count/len(predictions)
  return accuracy*100

# Reading and Filtering Filtered CNN features

In [7]:
url_filtered_CNN_features_dataset = 'https://raw.githubusercontent.com/AgarwalMayank2/Face_Detection/refs/heads/main/processed_dataset/filtered_CNN_features_dataset.csv' #for CNN limited
df = pd.read_csv(url_filtered_CNN_features_dataset) # reading url for extracted CNN_features_dataset_limited.csv
df.shape

(4324, 2050)

In [8]:
label_counts = df['2048'].value_counts()
valid_labels = label_counts[label_counts >= 80].index
filtered_df_cnn = df[df['2048'].isin(valid_labels)]
filtered_df_cnn = filtered_df_cnn.groupby('2048').apply(lambda x: x.sample(min(len(x), 105), random_state=42)).reset_index(drop=True)
    # limiting dataset to have max 105 images pre prson
filtered_df_cnn.drop('Unnamed: 0', axis=1, inplace=True)
filtered_df_cnn.head()

<ipython-input-8-198960730b12>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df_cnn = filtered_df_cnn.groupby('2048').apply(lambda x: x.sample(min(len(x), 105), random_state=42)).reset_index(drop=True)


,0,1,2,3,4,5,6,7,8,9,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048
0,0.250257,1.217750,0.665482,0.149379,0.585579,0.131689,3.068458,0.325341,0.206950,0.436043,...,0.245859,0.184804,0.412334,1.423770,0.350464,1.109347,0.022752,0.067753,0.214193,Colin_Powell
1,0.417056,0.858468,0.789033,0.118489,1.028153,0.022634,2.594352,0.231926,0.186995,0.144153,...,0.121101,0.164805,0.646671,0.894974,0.005102,1.088958,0.008620,0.020627,0.029414,Colin_Powell
2,0.289682,1.073486,1.232232,0.139371,0.700671,0.217070,3.823237,0.228988,0.164996,0.076503,...,0.082875,0.064406,0.632786,1.607113,0.020582,1.049611,0.063771,0.081492,0.263031,Colin_Powell
3,0.379367,0.771178,1.113026,0.063733,0.513514,0.030872,3.027960,0.096166,0.110920,0.207713,...,0.179462,0.181224,0.600749,1.382739,0.039597,1.023607,0.041599,0.125309,0.214228,Colin_Powell
4,0.331142,0.432032,1.176327,0.113665,0.561529,0.178007,3.026908,0.192866,0.051029,0.202801,...,0.081700,0.075439,0.814237,1.244029,0.054053,0.863067,0.034378,0.095257,0.301316,Colin_Powell


## Train/Test split - for CNN dataset

In [9]:
# Separate features and labels
X_cnn = filtered_df_cnn.iloc[:, :-1]
y_cnn = filtered_df_cnn.iloc[:, -1]

# Encode labels (alphabetically)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_cnn)
# encoding is benificial as working on numbers is lot easier than working on string

# Ensure stratified split (16 training, 4 testing per class)
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(X_cnn, y_encoded, test_size=1/5, random_state=42, stratify=y_encoded)

# Normalize features
scaler = StandardScaler()
X_train_cnn = scaler.fit_transform(X_train_cnn)
X_test_cnn = scaler.transform(X_test_cnn)

print(f"Dataset size: {filtered_df_cnn.shape}")
print(f"Training size: {X_train_cnn.shape}, Testing size: {X_test_cnn.shape}")

Dataset size: (525, 2049)
Training size: (420, 2048), Testing size: (105, 2048)


## Dimensionality reduction

In [10]:
from sklearn.decomposition import PCA

### Applying PCA on CNN features

In [13]:
pca=PCA(n_components=400)

X_train_cnn_pca=pca.fit_transform(X_train_cnn)
X_test_cnn_pca=pca.transform(X_test_cnn)

print(X_train_cnn_pca.shape)
print(X_test_cnn_pca.shape)

(420, 400)
(105, 400)


In [14]:
import numpy as np
print(np.sum(pca.explained_variance_ratio_))  # Should be close to 1

0.9988478889237746


### Applying LDA on CNN features

In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Apply LDA with at most (number of classes - 1) components
lda = LDA(n_components=min(500, len(set(y_train_cnn)) - 1))

X_train_cnn_lda = lda.fit_transform(X_train_cnn, y_train_cnn)
X_test_cnn_lda = lda.transform(X_test_cnn)

print(X_train_cnn_lda.shape)
print(X_test_cnn_lda.shape)

(420, 4)
(105, 4)


## Setting y_train and y_test

In [16]:
y_train = y_train_cnn
y_test = y_test_cnn

print(y_train[:10])
print(y_test[:10])

[4 2 4 1 3 1 2 2 1 3]
[1 1 2 1 1 3 2 1 3 0]


## Printing y_test

In [17]:
for i in range(len(y_test_cnn)):
  if i<=100:
    print(label_encoder.inverse_transform([y_test[i]])[0])

Donald_Rumsfeld
Donald_Rumsfeld
George_W_Bush
Donald_Rumsfeld
Donald_Rumsfeld
Gerhard_Schroeder
George_W_Bush
Donald_Rumsfeld
Gerhard_Schroeder
Colin_Powell
George_W_Bush
Tony_Blair
George_W_Bush
Tony_Blair
Gerhard_Schroeder
Tony_Blair
Tony_Blair
Gerhard_Schroeder
Tony_Blair
Gerhard_Schroeder
Tony_Blair
Colin_Powell
Tony_Blair
George_W_Bush
Colin_Powell
George_W_Bush
Gerhard_Schroeder
Gerhard_Schroeder
Donald_Rumsfeld
Colin_Powell
Tony_Blair
Tony_Blair
Donald_Rumsfeld
Tony_Blair
George_W_Bush
Colin_Powell
George_W_Bush
George_W_Bush
Gerhard_Schroeder
Tony_Blair
Donald_Rumsfeld
George_W_Bush
Tony_Blair
Donald_Rumsfeld
Donald_Rumsfeld
Gerhard_Schroeder
Tony_Blair
George_W_Bush
George_W_Bush
Tony_Blair
Donald_Rumsfeld
Gerhard_Schroeder
Tony_Blair
George_W_Bush
Tony_Blair
Gerhard_Schroeder
Gerhard_Schroeder
Gerhard_Schroeder
George_W_Bush
Donald_Rumsfeld
Donald_Rumsfeld
Donald_Rumsfeld
George_W_Bush
Tony_Blair
Donald_Rumsfeld
Colin_Powell
Colin_Powell
Colin_Powell
George_W_Bush
Gerhard_Sch

# Reading and Filtering Filtered LBP features

In [18]:
url_filtered_lbp_features = 'https://raw.githubusercontent.com/AgarwalMayank2/Face_Detection/refs/heads/main/processed_dataset/filtered_LBP_features_dataset.csv'
df = pd.read_csv(url_filtered_lbp_features)

In [19]:
label_counts = df['256'].value_counts()
valid_labels = label_counts[label_counts >= 80].index
filtered_df_LBP = df[df['256'].isin(valid_labels)]
filtered_df_LBP = filtered_df_LBP.groupby('256').apply(lambda x: x.sample(min(len(x), 105), random_state=42)).reset_index(drop=True)
    # limiting dataset to have max 105 images pre prson
filtered_df_LBP.drop('Unnamed: 0', axis=1, inplace=True)
print(filtered_df_LBP.head())

       0     1      2      3     4     5      6      7      8     9  ...  \
0  618.0  51.0  433.0  186.0  59.0  12.0  180.0  305.0  247.0  25.0  ...   
1  687.0  49.0  386.0  187.0  62.0   6.0  184.0  299.0  371.0  24.0  ...   
2  767.0  79.0  380.0  189.0  73.0   4.0  204.0  278.0  376.0  35.0  ...   
3  602.0  50.0  314.0  235.0  46.0   6.0  225.0  451.0  389.0  24.0  ...   
4  413.0  43.0  282.0  105.0  41.0  12.0   66.0  154.0  182.0  28.0  ...   

     247     248    249   250    251    252    253    254      255  \
0  325.0  1439.0  489.0  37.0  365.0  574.0  474.0  453.0  14110.0   
1  549.0  2319.0  662.0  40.0  550.0  586.0  474.0  443.0  14503.0   
2  429.0  2392.0  474.0  28.0  384.0  572.0  394.0  490.0  23504.0   
3  546.0  3026.0  863.0  55.0  766.0  721.0  497.0  564.0  11584.0   
4  306.0  2261.0  402.0  42.0  345.0  366.0  327.0  254.0  31421.0   

            256  
0  Colin_Powell  
1  Colin_Powell  
2  Colin_Powell  
3  Colin_Powell  
4  Colin_Powell  

[5 rows x 257

<ipython-input-19-f0ca6eb096e9>:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df_LBP = filtered_df_LBP.groupby('256').apply(lambda x: x.sample(min(len(x), 105), random_state=42)).reset_index(drop=True)


## Train/Test split - for LBP dataset

In [20]:
# Separate features and labels
X_LBP = filtered_df_LBP.iloc[:, :-1]
y_LBP = filtered_df_LBP.iloc[:, -1]

# Encode labels (alphabetically)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_LBP)
# encoding is benificial as working on numbers is lot easier than working on string

# Ensure stratified split (64 training, 16 testing per class)
X_train_LBP, X_test_LBP, y_train_LBP, y_test_LBP = train_test_split(X_LBP, y_encoded, test_size=1/5, random_state=42, stratify=y_encoded)

# Normalize features
scaler = StandardScaler()
X_train_LBP = scaler.fit_transform(X_train_LBP)
X_test_LBP = scaler.transform(X_test_LBP)

print(f"Dataset size: {filtered_df_LBP.shape}")
print(f"Training size: {X_train_LBP.shape}, Testing size: {X_test_LBP.shape}")

Dataset size: (525, 257)
Training size: (420, 256), Testing size: (105, 256)


## Dimensionality Reduction

### Applying PCA on LBP features

In [21]:
pca=PCA(n_components=150)

X_train_LBP_pca=pca.fit_transform(X_train_LBP)
X_test_LBP_pca=pca.transform(X_test_LBP)

print(X_train_LBP_pca.shape)
print(X_test_LBP_pca.shape)

(420, 150)
(105, 150)


In [22]:
import numpy as np
print(np.sum(pca.explained_variance_ratio_))  # Should be close to 1

0.9709552698602606


### Applying LDA on LBP features

In [23]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Apply LDA with at most (number of classes - 1) components
lda = LDA(n_components=min(500, len(set(y_train_LBP)) - 1))

X_train_LBP_lda = lda.fit_transform(X_train_LBP, y_train_LBP)
X_test_LBP_lda = lda.transform(X_test_LBP)

print(X_train_LBP_lda.shape)
print(X_test_LBP_lda.shape)

(420, 4)
(105, 4)


## Setting y_train and y_test

In [24]:
y_train = y_train_LBP
y_test = y_test_LBP

print(y_train[:10])
print(y_test[:10])

[4 2 4 1 3 1 2 2 1 3]
[1 1 2 1 1 3 2 1 3 0]


## Printing y_test for checking

In [25]:
for i in range(len(y_test_LBP)):
  if i<=100:
    print(label_encoder.inverse_transform([y_test[i]])[0])

Donald_Rumsfeld
Donald_Rumsfeld
George_W_Bush
Donald_Rumsfeld
Donald_Rumsfeld
Gerhard_Schroeder
George_W_Bush
Donald_Rumsfeld
Gerhard_Schroeder
Colin_Powell
George_W_Bush
Tony_Blair
George_W_Bush
Tony_Blair
Gerhard_Schroeder
Tony_Blair
Tony_Blair
Gerhard_Schroeder
Tony_Blair
Gerhard_Schroeder
Tony_Blair
Colin_Powell
Tony_Blair
George_W_Bush
Colin_Powell
George_W_Bush
Gerhard_Schroeder
Gerhard_Schroeder
Donald_Rumsfeld
Colin_Powell
Tony_Blair
Tony_Blair
Donald_Rumsfeld
Tony_Blair
George_W_Bush
Colin_Powell
George_W_Bush
George_W_Bush
Gerhard_Schroeder
Tony_Blair
Donald_Rumsfeld
George_W_Bush
Tony_Blair
Donald_Rumsfeld
Donald_Rumsfeld
Gerhard_Schroeder
Tony_Blair
George_W_Bush
George_W_Bush
Tony_Blair
Donald_Rumsfeld
Gerhard_Schroeder
Tony_Blair
George_W_Bush
Tony_Blair
Gerhard_Schroeder
Gerhard_Schroeder
Gerhard_Schroeder
George_W_Bush
Donald_Rumsfeld
Donald_Rumsfeld
Donald_Rumsfeld
George_W_Bush
Tony_Blair
Donald_Rumsfeld
Colin_Powell
Colin_Powell
Colin_Powell
George_W_Bush
Gerhard_Sch

# Calling KNN function on CNN and LBP features (without Dimensionality Reduction)

In [26]:
y_pred, acc = knn(5, X_train_cnn, X_test_cnn, y_test)
# print("Predictions : ", y_pred)
print("Test Accuracy : ", acc, " %")
y_pred, acc = knn(5, X_train_cnn, X_train_cnn, y_train)
# print("Predictions : ", y_pred)
print("Train Accuracy : ", acc, " %")

Label 0 detected : Donald_Rumsfeld
Label 10 detected : Tony_Blair
Label 20 detected : George_W_Bush
Label 30 detected : Donald_Rumsfeld
Label 40 detected : Donald_Rumsfeld
Label 50 detected : Donald_Rumsfeld
Label 60 detected : Gerhard_Schroeder
Label 70 detected : Gerhard_Schroeder
Label 80 detected : Colin_Powell
Label 90 detected : Tony_Blair
Label 100 detected : Colin_Powell
['Donald_Rumsfeld', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'George_W_Bush', 'Colin_Powell', 'Colin_Powell', 'Colin_Powell', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'Tony_Blair', 'Gerhard_Schroeder', 'Colin_Powell', 'Colin_Powell', 'George_W_Bush', 'Donald_Rumsfeld', 'Tony_Blair', 'Colin_Powell', 'Gerhard_Schroeder', 'Tony_Blair', 'Gerhard_Schroeder', 'Colin_Powell', 'Colin_Powell', 'Colin_Powell', 'Donald_Rumsfeld', 'Tony_Blair', 'Gerhard_Schroeder', 'Tony_Blair', 'George_W_Bush', 'Colin_Powell', 'Geo

In [27]:
y_pred, acc = knn(5, X_train_LBP, X_test_LBP, y_test)
print("Predictions : ", y_pred)
print("Test Accuracy : ", acc, " %")
y_pred, acc = knn(5, X_train_LBP, X_train_LBP, y_train)
print("Predictions : ", y_pred)
print("Train Accuracy : ", acc, " %")

Label 0 detected : Donald_Rumsfeld
Label 10 detected : Colin_Powell
Label 20 detected : Gerhard_Schroeder
Label 30 detected : Gerhard_Schroeder
Label 40 detected : Colin_Powell
Label 50 detected : Donald_Rumsfeld
Label 60 detected : Gerhard_Schroeder
Label 70 detected : George_W_Bush
Label 80 detected : Gerhard_Schroeder
Label 90 detected : Gerhard_Schroeder
Label 100 detected : Donald_Rumsfeld
['Donald_Rumsfeld', 'Tony_Blair', 'George_W_Bush', 'Donald_Rumsfeld', 'Colin_Powell', 'George_W_Bush', 'Gerhard_Schroeder', 'Tony_Blair', 'Colin_Powell', 'Colin_Powell', 'Colin_Powell', 'Tony_Blair', 'George_W_Bush', 'Donald_Rumsfeld', 'Colin_Powell', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'George_W_Bush', 'Gerhard_Schroeder', 'George_W_Bush', 'Gerhard_Schroeder', 'Tony_Blair', 'Tony_Blair', 'Colin_Powell', 'Tony_Blair', 'Donald_Rumsfeld', 'George_W_Bush', 'Tony_Blair', 'Colin_Powell', 'Gerhard_Schroeder', 'Gerhard_Schroeder', 'Tony_Blair', 'Donald_Rumsfeld', 'George_W_Bush', 'Tony_Blair', 'Ger

# Calling KNN function on CNN and LBP features (with Dimensionality Reduction)

## PCA on CNN and LBP features

In [28]:
y_pred, acc = knn(5, X_train_cnn_pca, X_test_cnn_pca, y_test)
print("Predictions : ", y_pred)
print("Test Accuracy : ", acc, " %")
y_pred, acc = knn(5, X_train_cnn_pca, X_train_cnn_pca, y_train)
print("Train Accuracy : ", acc, " %")

Label 0 detected : Donald_Rumsfeld
Label 10 detected : Tony_Blair
Label 20 detected : George_W_Bush
Label 30 detected : Donald_Rumsfeld
Label 40 detected : Donald_Rumsfeld
Label 50 detected : Donald_Rumsfeld
Label 60 detected : Gerhard_Schroeder
Label 70 detected : Gerhard_Schroeder
Label 80 detected : Colin_Powell
Label 90 detected : Tony_Blair
Label 100 detected : Colin_Powell
['Donald_Rumsfeld', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'George_W_Bush', 'George_W_Bush', 'Colin_Powell', 'Colin_Powell', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'Tony_Blair', 'Gerhard_Schroeder', 'Colin_Powell', 'Colin_Powell', 'George_W_Bush', 'Donald_Rumsfeld', 'Tony_Blair', 'Colin_Powell', 'Gerhard_Schroeder', 'Tony_Blair', 'Gerhard_Schroeder', 'Colin_Powell', 'Colin_Powell', 'Colin_Powell', 'Donald_Rumsfeld', 'Tony_Blair', 'Gerhard_Schroeder', 'Tony_Blair', 'George_W_Bush', 'Colin_Powell', 'Ge

In [29]:
y_pred, acc = knn(5, X_train_LBP_pca, X_test_LBP_pca, y_test)
print("Predictions : ", y_pred)
print("Test Accuracy : ", acc, " %")
y_pred, acc = knn(5, X_train_LBP_pca, X_train_LBP_pca, y_train)
print("Predictions : ", y_pred)
print("Train Accuracy : ", acc, " %")

Label 0 detected : Donald_Rumsfeld
Label 10 detected : Colin_Powell
Label 20 detected : Tony_Blair
Label 30 detected : Gerhard_Schroeder
Label 40 detected : Colin_Powell
Label 50 detected : Donald_Rumsfeld
Label 60 detected : Gerhard_Schroeder
Label 70 detected : George_W_Bush
Label 80 detected : George_W_Bush
Label 90 detected : Gerhard_Schroeder
Label 100 detected : Donald_Rumsfeld
['Donald_Rumsfeld', 'Tony_Blair', 'George_W_Bush', 'Donald_Rumsfeld', 'Colin_Powell', 'George_W_Bush', 'Gerhard_Schroeder', 'Tony_Blair', 'Colin_Powell', 'Colin_Powell', 'Colin_Powell', 'Tony_Blair', 'George_W_Bush', 'Donald_Rumsfeld', 'Colin_Powell', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Tony_Blair', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'Tony_Blair', 'Tony_Blair', 'Tony_Blair', 'Tony_Blair', 'Donald_Rumsfeld', 'George_W_Bush', 'Tony_Blair', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Gerhard_Schroeder', 'Tony_Blair', 'Donald_Rumsfeld', 'George_W_Bush', 'Tony_Blair', 'George_W_Bush', 'George_W_Bush

## LDA on CNN and LBP features

In [30]:
y_pred, acc = knn(5, X_train_cnn_lda, X_test_cnn_lda, y_test)
print("Predictions : ", y_pred)
print("Test Accuracy : ", acc, " %")
y_pred, acc = knn(5, X_train_cnn_lda, X_train_cnn_lda, y_train)
print("Predictions : ", y_pred)
print("Train Accuracy : ", acc, " %")

Label 0 detected : Donald_Rumsfeld
Label 10 detected : Tony_Blair
Label 20 detected : Tony_Blair
Label 30 detected : Colin_Powell
Label 40 detected : Donald_Rumsfeld
Label 50 detected : Donald_Rumsfeld
Label 60 detected : Donald_Rumsfeld
Label 70 detected : Gerhard_Schroeder
Label 80 detected : Colin_Powell
Label 90 detected : Colin_Powell
Label 100 detected : Donald_Rumsfeld
['Donald_Rumsfeld', 'Donald_Rumsfeld', 'George_W_Bush', 'Colin_Powell', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'George_W_Bush', 'Donald_Rumsfeld', 'Tony_Blair', 'Colin_Powell', 'Tony_Blair', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'Tony_Blair', 'Gerhard_Schroeder', 'Tony_Blair', 'Gerhard_Schroeder', 'Tony_Blair', 'Gerhard_Schroeder', 'George_W_Bush', 'George_W_Bush', 'Colin_Powell', 'Gerhard_Schroeder', 'Gerhard_Schroeder', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Colin_Powell', 'Colin_Powell', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'George_W_Bush', 'Colin_Powell', 'Georg

In [31]:
y_pred, acc = knn(5, X_train_LBP_lda, X_test_LBP_lda, y_test)
print("Predictions : ", y_pred)
print("Test Accuracy : ", acc, " %")
y_pred, acc = knn(5, X_train_LBP_lda, X_train_LBP_lda, y_train)
print("Predictions : ", y_pred)
print("Train Accuracy : ", acc, " %")

Label 0 detected : Donald_Rumsfeld
Label 10 detected : Colin_Powell
Label 20 detected : Gerhard_Schroeder
Label 30 detected : Donald_Rumsfeld
Label 40 detected : Gerhard_Schroeder
Label 50 detected : George_W_Bush
Label 60 detected : Donald_Rumsfeld
Label 70 detected : Tony_Blair
Label 80 detected : Colin_Powell
Label 90 detected : Gerhard_Schroeder
Label 100 detected : Gerhard_Schroeder
['Donald_Rumsfeld', 'Tony_Blair', 'Colin_Powell', 'Tony_Blair', 'Colin_Powell', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Colin_Powell', 'Donald_Rumsfeld', 'Colin_Powell', 'Tony_Blair', 'Colin_Powell', 'Tony_Blair', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Tony_Blair', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Colin_Powell', 'Tony_Blair', 'Gerhard_Schroeder', 'George_W_Bush', 'Gerhard_Schroeder', 'Tony_Blair', 'Colin_Powell', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 

# Combining CNN and LBP features (Concatenating)

In [32]:
combined_X_train = np.concatenate((X_train_cnn, X_train_LBP), axis=1)
combined_X_test = np.concatenate((X_test_cnn, X_test_LBP), axis=1)

In [33]:
print("No. of CNN features + No. of LBP features = ", X_train_cnn.shape[1] + X_train_LBP.shape[1])
print("Total no. of features after concatenation = ", combined_X_train.shape[1])

No. of CNN features + No. of LBP features =  2304
Total no. of features after concatenation =  2304


In [34]:
print(combined_X_train.shape[1])
print(combined_X_test.shape[1])

2304
2304


## Applying PCA on Combined features

In [36]:
pca=PCA(n_components=400)

X_train_pca=pca.fit_transform(combined_X_train)
X_test_pca=pca.transform(combined_X_test)

print(X_train_pca.shape)
print(X_test_pca.shape)

(420, 400)
(105, 400)


## Applying LDA on Combined features

In [37]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Apply LDA with at most (number of classes - 1) components
lda = LDA(n_components=min(500, len(set(y_train)) - 1))

X_train_lda = lda.fit_transform(combined_X_train, y_train)
X_test_lda = lda.transform(combined_X_test)

print(X_train_lda.shape)
print(X_test_lda.shape)

(420, 4)
(105, 4)


# Calling KNN function on Combined features (without Dimensionality Reduction)

In [38]:
y_pred, acc = knn(5, combined_X_train, combined_X_test, y_test)
print("Predictions : ", y_pred)
print("Test Accuracy : ", acc, " %")
y_pred, acc = knn(5, combined_X_train, combined_X_train, y_train)
print("Predictions : ", y_pred)
print("Train Accuracy : ", acc, " %")

Label 0 detected : Donald_Rumsfeld
Label 10 detected : Tony_Blair
Label 20 detected : George_W_Bush
Label 30 detected : Tony_Blair
Label 40 detected : Tony_Blair
Label 50 detected : Donald_Rumsfeld
Label 60 detected : Gerhard_Schroeder
Label 70 detected : Gerhard_Schroeder
Label 80 detected : Colin_Powell
Label 90 detected : Tony_Blair
Label 100 detected : Gerhard_Schroeder
['Donald_Rumsfeld', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'George_W_Bush', 'George_W_Bush', 'Colin_Powell', 'Colin_Powell', 'Tony_Blair', 'Tony_Blair', 'Tony_Blair', 'Tony_Blair', 'George_W_Bush', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Gerhard_Schroeder', 'Colin_Powell', 'Tony_Blair', 'George_W_Bush', 'Donald_Rumsfeld', 'Tony_Blair', 'Tony_Blair', 'Gerhard_Schroeder', 'Tony_Blair', 'Gerhard_Schroeder', 'Colin_Powell', 'Donald_Rumsfeld', 'Colin_Powell', 'Tony_Blair', 'Tony_Blair', 'Gerhard_Schroeder', 'Tony_Blair', 'Tony_Blair', 'Colin_Powell', 'Gerhard_Sch

# Calling KNN function on Combined features (with Dimensionality Reduction)

## PCA on Combined features

In [39]:
y_pred, acc = knn(5, X_train_pca, X_test_pca, y_test)
print("Predictions : ", y_pred)
print("Test Accuracy : ", acc, " %")
y_pred, acc = knn(5, X_train_pca, X_train_pca, y_train)
print("Predictions : ", y_pred)
print("Train Accuracy : ", acc, " %")

Label 0 detected : Donald_Rumsfeld
Label 10 detected : Tony_Blair
Label 20 detected : George_W_Bush
Label 30 detected : Tony_Blair
Label 40 detected : Tony_Blair
Label 50 detected : Donald_Rumsfeld
Label 60 detected : Colin_Powell
Label 70 detected : Gerhard_Schroeder
Label 80 detected : Colin_Powell
Label 90 detected : Tony_Blair
Label 100 detected : Gerhard_Schroeder
['Donald_Rumsfeld', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'George_W_Bush', 'George_W_Bush', 'Colin_Powell', 'Colin_Powell', 'Tony_Blair', 'Tony_Blair', 'Tony_Blair', 'Tony_Blair', 'George_W_Bush', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Gerhard_Schroeder', 'Colin_Powell', 'Tony_Blair', 'George_W_Bush', 'Colin_Powell', 'Tony_Blair', 'Tony_Blair', 'Gerhard_Schroeder', 'Tony_Blair', 'Gerhard_Schroeder', 'Colin_Powell', 'Donald_Rumsfeld', 'Colin_Powell', 'Tony_Blair', 'Tony_Blair', 'Gerhard_Schroeder', 'Tony_Blair', 'Tony_Blair', 'Colin_Powell', 'Gerhard_Schroeder',

## LDA on Combined features

In [40]:
y_pred, acc = knn(5, X_train_lda, X_test_lda, y_test)
print("Predictions : ", y_pred)
print("Test Accuracy : ", acc, " %")
y_pred, acc = knn(5, X_train_lda, X_train_lda, y_train)
print("Predictions : ", y_pred)
print("Train Accuracy : ", acc, " %")

Label 0 detected : Donald_Rumsfeld
Label 10 detected : Tony_Blair
Label 20 detected : Tony_Blair
Label 30 detected : Tony_Blair
Label 40 detected : Donald_Rumsfeld
Label 50 detected : Donald_Rumsfeld
Label 60 detected : Donald_Rumsfeld
Label 70 detected : Gerhard_Schroeder
Label 80 detected : Colin_Powell
Label 90 detected : Colin_Powell
Label 100 detected : Donald_Rumsfeld
['Donald_Rumsfeld', 'Donald_Rumsfeld', 'George_W_Bush', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'George_W_Bush', 'Donald_Rumsfeld', 'Tony_Blair', 'Colin_Powell', 'Tony_Blair', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'Tony_Blair', 'Gerhard_Schroeder', 'Tony_Blair', 'Gerhard_Schroeder', 'Tony_Blair', 'Gerhard_Schroeder', 'George_W_Bush', 'George_W_Bush', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'Gerhard_Schroeder', 'Colin_Powell', 'Colin_Powell', 'Tony_Blair', 'Tony_Blair', 'George_W_Bush', 'Tony_Blair', 'George_W_Bush', 'Colin_Powell', 'George_W

# Visualization

## Apply LDA on Combined features to keep only 2 features

In [ ]:
lda = LDA(n_components=2)

X_train_lda_graph = lda.fit_transform(combined_X_train, y_train)
X_test_lda_graph = lda.transform(combined_X_test)

print(X_train_lda_graph.shape)
print(X_test_lda_graph.shape)

(420, 2)
(105, 2)


In [ ]:
y_pred, acc = knn(5, X_train_lda_graph, X_test_lda_graph, y_test)
print("Predictions : ", y_pred)
print("Accuracy : ", acc, " %")

Label 0 detected : Donald_Rumsfeld
Label 10 detected : George_W_Bush
Label 20 detected : Tony_Blair
Label 30 detected : Gerhard_Schroeder
Label 40 detected : Donald_Rumsfeld
Label 50 detected : Donald_Rumsfeld
Label 60 detected : Donald_Rumsfeld
Label 70 detected : Gerhard_Schroeder
Label 80 detected : Colin_Powell
Label 90 detected : Gerhard_Schroeder
Label 100 detected : Donald_Rumsfeld
['Donald_Rumsfeld', 'Donald_Rumsfeld', 'George_W_Bush', 'Colin_Powell', 'Donald_Rumsfeld', 'Gerhard_Schroeder', 'George_W_Bush', 'Donald_Rumsfeld', 'George_W_Bush', 'Colin_Powell', 'George_W_Bush', 'Tony_Blair', 'Colin_Powell', 'Tony_Blair', 'Gerhard_Schroeder', 'George_W_Bush', 'George_W_Bush', 'Donald_Rumsfeld', 'George_W_Bush', 'Gerhard_Schroeder', 'Tony_Blair', 'Gerhard_Schroeder', 'George_W_Bush', 'George_W_Bush', 'Gerhard_Schroeder', 'Donald_Rumsfeld', 'Donald_Rumsfeld', 'Colin_Powell', 'Gerhard_Schroeder', 'Colin_Powell', 'Gerhard_Schroeder', 'Tony_Blair', 'Gerhard_Schroeder', 'George_W_Bush', 

In [ ]:
# Actual labels
y_test_labels = label_encoder.inverse_transform(y_test)

In [ ]:
print(np.unique(y_test_labels))

['Colin_Powell' 'Donald_Rumsfeld' 'George_W_Bush' 'Gerhard_Schroeder'
 'Tony_Blair']


### Graph visualisation

<ipython-input-90-310e31f358de>:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(classes))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

X_plot = X_test_lda_graph
y_actual = np.array(y_test_labels)
y_pred = np.array(y_pred)

classes = np.unique(y_actual)
colors = plt.cm.get_cmap('tab10', len(classes))

# Identify misclassified points
misclassified = y_actual != y_pred

# Create side-by-side subplots
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# --- Plot y_actual ---
for i, label in enumerate(classes):
    indices = np.where(y_actual == label)
    axs[0].scatter(X_plot[indices, 0], X_plot[indices, 1], label=f'{label}', color=colors(i))
axs[0].set_title("Actual Labels (y_actual)")
axs[0].set_xlabel("LDA Component 1")
axs[0].set_ylabel("LDA Component 2")
axs[0].legend()
axs[0].grid(True)

# --- Plot y_pred ---
for i, label in enumerate(classes):
    indices = np.where(y_pred == label)
    axs[1].scatter(X_plot[indices, 0], X_plot[indices, 1], label=f'{label}', color=colors(i), alpha=0.7)

# Highlight misclassified points with red borders
axs[1].scatter(X_plot[misclassified, 0], X_plot[misclassified, 1],
               facecolors='none', edgecolors='red', s=100, linewidths=1.5, label='Misclassified')

axs[1].set_title("Predicted Labels (y_pred)")
axs[1].set_xlabel("LDA Component 1")
axs[1].set_ylabel("LDA Component 2")
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()


### Confusion Matrices (One vs All)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Inputs
y_true = np.array(y_test_labels)
y_pred = np.array(y_pred)

classes = np.unique(y_true)
class_names = [f"{c}" for c in classes]  # or actual names

n_classes = len(classes)
cols = 5  # All in one row
rows = (n_classes + cols - 1) // cols

# Slightly bigger square plots
fig, axes = plt.subplots(rows, cols, figsize=(cols * 2.8, rows * 2.8), squeeze=False)
fig.suptitle(" 'One vs All' Confusion Matrices", fontsize=15, fontweight='bold')

for i, cls in enumerate(classes):
    row, col = divmod(i, cols)
    ax = axes[row][col]

    binary_true = (y_true == cls).astype(int)
    binary_pred = (y_pred == cls).astype(int)
    cm = confusion_matrix(binary_true, binary_pred)

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=ax,
                square=True, xticklabels=["No", "Yes"], yticklabels=["No", "Yes"],
                annot_kws={"size": 12})

    ax.set_title(class_names[i], fontsize=11, fontweight='semibold')
    ax.set_xlabel("Predicted", fontsize=9)
    ax.set_ylabel("Actual", fontsize=9)
    ax.tick_params(labelsize=9)

# Turn off extra axes
for j in range(i+1, rows*cols):
    row, col = divmod(j, cols)
    axes[row][col].axis("off")

plt.tight_layout(rect=[0, 0, 1, 0.92])
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Inputs
y_true = np.array(y_test_labels)
y_pred = np.array(y_pred)

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Define class names
classes = np.unique(y_true)
class_names = [f"{c}" for c in classes]  # or actual names if you have them

# Plot the confusion matrix
plt.figure(figsize=(7, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            square=True, annot_kws={"size": 12})

plt.title("Confusion Matrix", fontsize=15, fontweight='bold')
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Use your LDA-reduced data
X_train_plot = X_train_lda_graph
X_test_plot = X_test_lda_graph

# Convert labels to integers for coloring
y_train_int = label_encoder.transform(y_train)

# Define mesh grid boundaries
x_min, x_max = X_train_plot[:, 0].min() - 1, X_train_plot[:, 0].max() + 1
y_min, y_max = X_train_plot[:, 1].min() - 1, X_train_plot[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05),
                     np.arange(y_min, y_max, 0.05))

# Predict over the entire mesh grid using your KNN function
grid_points = np.c_[xx.ravel(), yy.ravel()]
grid_preds = []

print("Generating decision boundary...")

for point in grid_points:
    # Use your custom KNN with just one test point at a time
    pred, _ = knn(5, X_train_plot, X_test=np.array([point]), y_test=[0])  # dummy y_test
    grid_preds.append(label_encoder.transform([pred[0]])[0])  # convert back to int

Z = np.array(grid_preds).reshape(xx.shape)

# Plot decision boundary
plt.figure(figsize=(10, 6))
cmap_light = ListedColormap(['#FFCCCC', '#CCFFCC', '#CCCCFF', '#FFFFCC', '#E0CCFF'])
cmap_bold = ListedColormap(['#FF0000', '#00CC00', '#0000FF', '#CCCC00', '#8000FF'])

plt.contourf(xx, yy, Z, cmap=cmap_light, alpha=0.6)

# Plot training points
scatter = plt.scatter(X_train_plot[:, 0], X_train_plot[:, 1], c=y_train_int, cmap=cmap_bold, edgecolor='k', s=60)
plt.xlabel("LDA Component 1")
plt.ylabel("LDA Component 2")
plt.title("KNN Decision Boundary (Custom KNN)")

# Legend
handles = scatter.legend_elements()[0]
labels = label_encoder.inverse_transform(np.unique(y_train_int))
plt.legend(handles, labels, title="Classes", loc="upper right")

plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()


ValueError: y contains previously unseen labels: 2